<img src="https://www.utpl.edu.ec/manual_imagen/images/institucional/UTPL-INSTITUCIONAL-color.jpg" style = "width:510px;height:180px">

<H1 style="text-align: center;">Plotly Widgets</H1>

<p style="text-align: justify;">Plotly widgets representa una forma interactiva de analizar datos que son obtenidos desde diferentes orígenes, funciona en base a la librerìa de visualización de datos Plotly y hace uso de los siguientes recursos para su ejecución</p>

<H1 style="text-align: left;">Generación de Gráficas</H1>

<p style="text-align: justify;">Para la generación de cualquier gráfica, necesitamos hacer uso de librerías que nos permiten utilizar los diferentes componentes de Plotly en este caso</p>

In [1]:
import datetime
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets
import chart_studio.plotly as py
from plotly.graph_objs import Scatter, Layout, Figure, Data, Stream, YAxis, Marker

<p style="text-align: justify;">Para acceder a la colección almacenada en MongoDB requerimos hacer uso de las siguientes librerías</p>

In [2]:
import os # crear interfaz
import sys # conexion a host
import pymongo # API MongoDB
import pandas as pd
from pymongo import MongoClient

<p style="text-align: justify;">Conexión a cliente MongoDB</p>

In [3]:
mongoClient = MongoClient('localhost',27017) #Conexión a localHost
db = mongoClient.gp #Base de datos GP
collection = db.vuelos #Colección Vuelos

<p style="text-align: justify;">Conexión a Plotly</p>

In [4]:
py.sign_in('lderazo1', 'TtDeC4A54WSSm0pNWMD8')

<p style="text-align: justify;">Generación DataFrame</p>

In [5]:
df = pd.DataFrame(list(collection.find()))

<p style="text-align: justify;">El siguiente archivo hace referencia a los vuelos realizados en estados unidos, especìficamente en New York durante el año 2013</p>

<p style="text-align: justify;">Inicializamos y limitamos los componentes de los cuales haremos uso para la generaciòn de la gráfica</p>

In [6]:
df.head()

,_id,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,5e45b7fc5f6b5037bf9b4cbe,2013,1,1,517,2,830,11,UA,N14228,1545,EWR,IAH,227,1400,5,17
1,5e45b7fc5f6b5037bf9b4cbf,2013,1,1,533,4,850,20,UA,N24211,1714,LGA,IAH,227,1416,5,33
2,5e45b7fc5f6b5037bf9b4cc0,2013,1,1,542,2,923,33,AA,N619AA,1141,JFK,MIA,160,1089,5,42
3,5e45b7fc5f6b5037bf9b4cc1,2013,1,1,544,-1,1004,-18,B6,N804JB,725,JFK,BQN,183,1576,5,44
4,5e45b7fc5f6b5037bf9b4cc2,2013,1,1,554,-6,812,-25,DL,N668DN,461,LGA,ATL,116,762,5,54


<p style="text-align: justify;"><b>Fuente: </b>GitHub [Conjunto de datos]. Tomado de usuario Yankev.<br><b>URL: </b><a>https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv/</a></p>

<H1 style="text-align: left;">Creación de Componentes</H1>

<p style="text-align: justify;">Generación de un Slider en base a los meses del año</p>

In [7]:
mes = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Mes:',
    continuous_update=False
)

<p style="text-align: justify;">Generación de un CheckBox, en este caso será para que el usuario controle si gusta filtrar o no por fecha la cantidad de vuelos</p>

In [8]:
fecha = widgets.Checkbox(
    description='Fecha: ',
    value=True,
)

<p style="text-align: justify;">Generación del contenedor superior que controlará tanto la fecha como los meses de vuelos</p>

In [9]:
contUno = widgets.HBox(children=[fecha, mes])

<p style="text-align: justify;">Se genera las listas despegables, mediante los campos unicos que formarán parte de nuestra lista de filtrado, en este caso Aereolineas</p>

In [10]:
aerolinea = widgets.Dropdown(
    description='Aereolinea:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

<p style="text-align: justify;">Se genera las listas despegables, mediante los campos unicos que formarán parte de nuestra lista de filtrado, en este caso Aereopuerto de Origen</p>

In [20]:
origen = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Aereopuerto de Origen:',
)

<p style="text-align: justify;">Generación de layout vacío, y especificación del tipo de gráfica a representar (Histogramas de retraso en llegada y salida)</p>

In [21]:
histUno = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Retraso en llegada')
histDos = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Retraso en salida')
g = go.FigureWidget(data=[histUno, histDos],layout = go.Layout(title=dict(text='Base de datos de vuelos en la ciudad de Nueva York'),barmode='overlay'))

<p style="text-align: justify;">Validación que es realizada con el fin de verificar los campos ùnicos</p>

In [22]:
def validacion():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False

<p style="text-align: justify;">En este caso, tanto los mètodos de validación como de respuesta son llamados una vez el usuario interactùa con los componentes de la gráfica (checkbox, slider y dropDown list)</p>

In [23]:
def respuesta(change):
    if validacion():
        if fecha.value:
            filter_list = [i and j and k for i, j, k in zip(df['month'] == mes.value, df['carrier'] == aerolinea.value, df['origin'] == origen.value)]
            temp_df = df[filter_list]

        else:
            filter_list = [i and j for i, j in zip(df['carrier'] == 'DL', df['origin'] == origen.value)]
            temp_df = df[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Retraso en minutos'
            g.layout.yaxis.title = 'Numero de retrasos'

<p style="text-align: justify;">Mediante observe, el componente queda expectante a que el usuario interactùe en la gráfica, de esta forma se generan los valores que pueden ser visualizados en tiempo real, otro aspecto a tomar en cuenta es que si el usuario decide no interactuar en el sistema se muestran los valores por defecto que le corresponden al aereopuerto LGA y la aereolìnea DL.</p>

In [24]:
origen.observe(respuesta, names="value")
aerolinea.observe(respuesta, names="value")
mes.observe(respuesta, names="value")
fecha.observe(respuesta, names="value")

In [26]:
contDos = widgets.HBox([origen, aerolinea])
widgets.VBox([contUno,contDos,g])

<img src="https://cloud.githubusercontent.com/assets/12302455/16637308/4e476280-43ac-11e6-9fd3-ada2c9506ee1.gif">